In [1]:
#03_summarize_every_results
#
#leverage results from *.padj.tsv* (created by "02_calculated_BH_pvalues")
#   - simply adding foldchange values

import pandas as pd
import statistics
import math

In [2]:
analysis_result_dir = '../../../analysis/statistics/linear_model/differential_abundance_v2/'
patient_info_file = '../../../preprocessed_data/meta/patient_info.ML_ready.tsv'

comparison_list = ['cVSneg', 'cVSpos','negVSpos']
condition_list = ['autoantibody','metabolomics','proteomics']

for condition in condition_list:
    
    if condition == 'metabolomics':
        data_profile_file = '../../../preprocessed_data/metabolomics/metabolone_raw_norm_preprocessed.v2.tsv'
    if condition == 'proteomics':
        data_profile_file = '../../../preprocessed_data/proteomics/somascan_anml.T.v2.tsv'
    if condition == 'autoantibody':
        data_profile_file = '../../../preprocessed_data/autoantibody/sengenics_qnorm_data.v2.tsv'
    
    feature_info_dict = {}

    for comparison in comparison_list:
        data_file = '%s%s.%s.padj.v2.tsv' % (analysis_result_dir, condition, comparison)
        data_df = pd.read_csv(data_file, sep="\t", index_col=0)
        feature_list = data_df.index.values
        r, c = data_df.shape
        for i in range(r):
            feature = feature_list[i]
            cohenD = data_df["cohenD"][i]
            pval = data_df["pval"][i]
            padj = data_df["padj"][i]
            fc = data_df["fc_case_control"][i]

            group_a = comparison.split('VS')[0]
            group_b = comparison.split('VS')[1]

            feature_info_dict[comparison, feature] = [cohenD, pval, padj, fc]
            
        #write first output for the record
        #condition.comparison.v2.tsv
        output_file = '%s%s.%s.padj.sig.tsv' % (analysis_result_dir, condition, comparison)
        output_txt = open(output_file,'w')
        output_txt.write('\tcohenD\tfc_case_control\tpval\tpadj\n')
        for feature in feature_list:
            
            cohenD = feature_info_dict[comparison, feature][0]
            pval = feature_info_dict[comparison, feature][1]
            padj = feature_info_dict[comparison, feature][2]
            fc = feature_info_dict[comparison, feature][3]

            if cohenD < -0.5 or cohenD > 0.5:
                
                if condition == "autoantibody":
                    if pval < 0.05:
                        output_txt.write('%s\t%s\t%s\t%s\t%s\n' % (feature, cohenD, fc, pval, padj))
                
                else:
                    if pval < 0.01:
                        output_txt.write('%s\t%s\t%s\t%s\t%s\n' % (feature, cohenD, fc, pval, padj))

        output_txt.close()


